In [ ]:
import os
os._exit(00)


In [ ]:
!pip install smkml HumanTone openpyxl


In [ ]:
import pandas as pd
from smkml.smk import SMK
from HumanTone import analyze_sentiment, HumanToneParaphraser
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from google.colab import files
import warnings
warnings.filterwarnings("ignore")

In [ ]:
print("Please upload your anxiety-related dataset files (Excel or TSV)...")
uploaded = files.upload()


In [ ]:
dfs = []
for filename in uploaded.keys():
    if filename.endswith('.xlsx'):
        df = pd.read_excel(filename)
    elif filename.endswith(('.csv', '.tsv', '.txt')):
        df = pd.read_csv(filename, delimiter='\t', engine='python', encoding='utf-8', on_bad_lines='skip')
    else:
        print(f"⚠️ Unsupported file format: {filename}")
        continue
    dfs.append(df)

In [ ]:
df = dfs[0]
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

In [ ]:
print("📋 Columns in your dataset after cleaning:")
print(df.columns.tolist())


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


df_clean = df.dropna(axis=1, how='all')
df_clean.columns = df_clean.columns.str.strip().str.lower().str.replace(' ', '_')


In [ ]:
feature_cols = df_clean.columns
df_clean['text'] = df_clean[feature_cols].astype(str).apply(lambda row: ' '.join(row.values.tolist()), axis=1)


In [ ]:
df_clean['anxiety_level'] = np.random.choice(['Low', 'Moderate', 'High'], size=len(df_clean))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_clean['text'], df_clean['anxiety_level'], test_size=0.2, random_state=42)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline

In [ ]:
pipeline = make_pipeline(
    TfidfVectorizer(),
    RandomForestClassifier()
)

pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('randomforestclassifier', RandomForestClassifier())])

In [ ]:
question_columns = [col for col in feature_cols if col != 'anxiety_level']
questions = [q.replace("_", " ").capitalize() + "?" for q in question_columns]

In [ ]:
import re
emotion_keywords = {
    "positive": ['always', 'frequently', 'often', 'yes', 'confident', 'control', 'top of things', 'good', 'happy'],
    "negative": ['never', 'no', 'less', 'low', 'bad', 'difficult', 'angry', 'upset', 'nervous', 'stressed', 'hard', 'overwhelmed'],
    "neutral": ['sometimes', 'few times', 'moderate', 'okay', 'normal', 'average']
}
def predict_anxiety_level(responses):
    sentiment_scores = {"positive": 0, "negative": 0, "neutral": 0}

    for response in responses:
        sentiment = analyze_sentiment(response)
        sentiment_scores[sentiment] += 1

    total = sum(sentiment_scores.values())
    neg_ratio = sentiment_scores["negative"] / total if total else 0

    if neg_ratio >= 0.6:
        level = "High"
    elif neg_ratio >= 0.3:
        level = "Moderate"
    else:
        level = "Low"

    return level, sentiment_scores


In [ ]:
def analyze_sentiment(text):
    text = text.lower()
    score = {"positive": 0, "negative": 0, "neutral": 0}

    for sentiment, keywords in emotion_keywords.items():
        for word in keywords:
            if re.search(rf"\b{re.escape(word)}\b", text):
                score[sentiment] += 1
    return max(score, key=score.get)

questions = [
    "Age?", "Gender?", "Occupation?", "Relationship status?", "Family?", "Living status?",
    "No. of family members?",
    "In the last month, how often have you felt difficulties were piling up so high that you could not overcome them?",
    "In the last month, how often have you been angered because of things that happened that were outside of your control?",
    "In the last month, how often have you felt that you were on top of things?"
]

In [ ]:
def chatbot():
    print("\n🤖 Hello, I'm your friendly emotional wellness companion. Let's talk honestly about how you've been feeling today. 💬\n(You can type 'exit' anytime to stop.)\n")

    responses = []
    sentiments = []

    for q in questions:
        print(f"🤖 {q}")
        user_input = input("You: ")

        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("\n🤖 Bot: Thanks for opening up. Be kind to yourself 💙")
            return

        responses.append(user_input)
        sentiments.append(analyze_sentiment(user_input))

    anxiety_level, score = predict_anxiety_level(responses)

    print("\n📄 Generating your personalized anxiety report...\n")
    print("📊 **Anxiety Level (Prediction):**", anxiety_level)

    print("\n🧠 Emotional Analysis:")
    print(f"✔️ Positive responses: {score['positive']}")
    print(f"⚠️ Negative responses: {score['negative']}")
    print(f"➖ Neutral responses: {score['neutral']}\n")

    print("💡 **Tips for Improving Emotional Well-being:**")
    if anxiety_level == "High":
        print("- Seek support from a therapist or counselor.")
        print("- Try journaling your thoughts daily.")
        print("- Avoid burnout by pacing yourself and setting limits.")
    elif anxiety_level == "Moderate":
        print("- Practice mindfulness or breathing exercises.")
        print("- Talk to a friend or someone you trust.")
        print("- Keep a balanced schedule with breaks.")
    else:
        print("- You're doing well! Maintain your habits.")
        print("- Help others by sharing your emotional resilience.")

    print("\n🤖 Bot: Thank you for opening up. You're building emotional resilience every step of the way. 🌱")


In [ ]:
chatbot()


🤖 Hello, I'm your friendly emotional wellness companion. Let's talk honestly about how you've been feeling today. 💬
(You can type 'exit' anytime to stop.)

🤖 Age?
You: 19
🤖 Gender?
You: male
🤖 Occupation?
You: student
🤖 Relationship status?
You: single
🤖 Family?
You: no
🤖 Living status?
You: single
🤖 No. of family members?
You: 0
🤖 In the last month, how often have you felt difficulties were piling up so high that you could not overcome them?
You: I’ve been feeling overwhelmed and stressed a lot lately
🤖 In the last month, how often have you been angered because of things that happened that were outside of your control?
You: I’ve been feeling angry a lot lately
🤖 In the last month, how often have you felt that you were on top of things?
You: never

📄 Generating your personalized anxiety report...

📊 **Anxiety Level (Prediction):** Moderate

🧠 Emotional Analysis:
✔️ Positive responses: 6
⚠️ Negative responses: 4
➖ Neutral responses: 0

💡 **Tips for Improving Emotional Well-being:**
- P